In [1]:
%matplotlib auto
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import math
from mpl_toolkits.mplot3d import Axes3D

Using matplotlib backend: <object object at 0x00000159140E8B50>


### NTU60的动作

In [2]:
action_names = ['drink water 1', 'eat meal/snack 2', 'brushing teeth 3', 'brushing hair 4', 'drop 5', 'pickup 6',
            'throw 7', 'sitting down 8', 'standing up 9', 'clapping 10', 'reading 11', 'writing 12',
            'tear up paper 13', 'wear jacket 14', 'take off jacket 15', 'wear a shoe 16', 'take off a shoe 17',
            'wear on glasses 18','take off glasses 19', 'put on a hat/cap 20', 'take off a hat/cap 21', 'cheer up 22',
            'hand waving 23', 'kicking something 24', 'put/take out sth 25', 'hopping 26', 'jump up 27',
            'make a phone call 28', 'playing with a phone 29', 'typing on a keyboard 30',
            'pointing to sth with finger 31', 'taking a selfie 32', 'check time (from watch) 33',
            'rub two hands together 34', 'nod head/bow 35', 'shake head 36', 'wipe face 37', 'salute 38',
            'put the palms together 39', 'cross hands in front 40', 'sneeze/cough 41', 'staggering 42', 'falling 43',
            'touch head 44', 'touch chest 45', 'touch back 46', 'touch neck 47', 'nausea or vomiting condition 48',
            'use a fan 49', 'punching 50', 'kicking other person 51', 'pushing other person 52',
            'pat on back of other person 53', 'point finger at the other person 54', 'hugging other person 55',
            'giving sth to other person 56', 'touch other person pocket 57', 'handshaking 58',
            'walking towards each other 59', 'walking apart from each other 60']
max_frame = 300
max_body = 2
max_joint = 25

### 读取数据

#### 1.读取skeleton文件

In [3]:
def read_skeleton(file):
    with open(file, 'r') as f:
        skeleton_sequence = {}
        skeleton_sequence['numFrame'] = int(f.readline())
        skeleton_sequence['frameInfo'] = []
        for t in range(skeleton_sequence['numFrame']):
            frame_info = {}
            frame_info['numBody'] = int(f.readline())
            frame_info['bodyInfo'] = []
            for m in range(frame_info['numBody']):
                body_info = {}
                body_info_key = [
                    'bodyID', 'clipedEdges', 'handLeftConfidence',
                    'handLeftState', 'handRightConfidence', 'handRightState',
                    'isResticted', 'leanX', 'leanY', 'trackingState'
                ]
                body_info = {
                    k: float(v)
                    for k, v in zip(body_info_key, f.readline().split())
                }
                body_info['numJoint'] = int(f.readline())
                body_info['jointInfo'] = []
                for v in range(body_info['numJoint']):
                    joint_info_key = [
                        'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                        'orientationW', 'orientationX', 'orientationY',
                        'orientationZ', 'trackingState'
                    ]
                    joint_info = {
                        k: float(v)
                        for k, v in zip(joint_info_key, f.readline().split())
                    }
                    body_info['jointInfo'].append(joint_info)
                frame_info['bodyInfo'].append(body_info)
            skeleton_sequence['frameInfo'].append(frame_info)
    return skeleton_sequence

#### 2.读取60个样本，每个样本归属于一类动作，共60类

In [4]:
data_file = "../data/ntu60/c003"
data = np.zeros((60, 3,max_frame, max_joint, max_body))   # 60个不同动作的样本坐标
frame_nums = np.zeros((60))
for file in os.listdir(data_file):
    action_class = int(file[file.find('A')+1:file.find('A')+4])   # 动作编号
    action_name = action_names[action_class-1]    # 动作名称
    file_path = os.path.join(data_file, file)    # skeleton文件路径
    skeleton_info = read_skeleton(file_path)   # 读取一个skeleton文件信息
    frame_nums[action_class-1] = skeleton_info['numFrame']
    for n, f in enumerate(skeleton_info['frameInfo']):
        for m, b in enumerate(f['bodyInfo']):
            for j, v in enumerate(b['jointInfo']):
                if m < max_body and j < max_joint:
                    data[action_class-1, :, n, j, m] = [v['x'], v['y'], v['z']]
                else:
                    pass

In [5]:
print(data.shape)
print(frame_nums)

(60, 3, 300, 25, 2)
[ 98.  93. 121. 130.  60.  83.  59.  70.  59.  64. 105. 118.  84. 128.
 186. 125.  99. 161.  84.  88.  85.  81.  59.  66.  90.  99.  60. 106.
 114. 142.  52.  84.  71.  77.  80.  71. 105.  87.  73.  53.  74. 101.
  76.  80.  84.  98.  82. 128. 106.  59.  68.  63.  88.  62.  55.  68.
  91.  71.  87.  76.]


### 开始绘制骨骼图

#### 1.旋转操作

In [6]:
def rotation_xyz(data, angle):
    # 旋转数据操作
    # ntu沿y轴
    rotation_matrix_y = np.array([[math.cos(angle), 0, -math.sin(angle)],
                                  [0, 1, 0],
                                  [math.sin(angle), 0, math.cos(angle)]])
    c,j,m = data.shape
    data = data.reshape((c,j*m))
    data = np.dot(rotation_matrix_y,data)
    data = data.reshape((c,j,m))
    return data

#### 2.对旋转后的样本骨骼范围进行测算

In [7]:
def skel_boundary(data):
    maxX, minX = data[0,:,0].max(), data[0,:, 0].min()
    maxY, minY = data[1,:, 0].max(), data[1,:, 0].min()
    return maxX, minX, maxY, minY

#### 3.判断是否满足四边形条件

In [8]:
def judge_quadrilateral(coordinate, occ1, occ2):
    world_coordinate = np.array(coordinate)
    # 遮挡操作
    occ1_maxX, occ1_minX, occ1_maxY, occ1_minY = occ1
    occ2_maxX, occ2_minX, occ2_maxY, occ2_minY = occ2
    occ1_vertex_a = np.array([occ1_minX, occ1_maxY, 2.6])
    occ1_vertex_b = np.array([occ1_maxX, occ1_maxY, 2.6])
    occ1_vertex_c = np.array([occ1_maxX, occ1_minY, 2.6])
    occ1_vertex_d = np.array([occ1_minX, occ1_minY, 2.6])
    z1 = np.dot((occ1_vertex_b - occ1_vertex_a), (world_coordinate - occ1_vertex_a))
    z2 = np.dot((occ1_vertex_c - occ1_vertex_b), (world_coordinate - occ1_vertex_b))
    z3 = np.dot((occ1_vertex_d - occ1_vertex_c), (world_coordinate - occ1_vertex_c))
    z4 = np.dot((occ1_vertex_a - occ1_vertex_d), (world_coordinate - occ1_vertex_d))

    occ2_vertex_a = np.array([occ2_minX, occ2_maxY, 2.6])
    occ2_vertex_b = np.array([occ2_maxX, occ2_maxY, 2.6])
    occ2_vertex_c = np.array([occ2_maxX, occ2_minY, 2.6])
    occ2_vertex_d = np.array([occ2_minX, occ2_minY, 2.6])
    z5 = np.dot((occ2_vertex_b - occ2_vertex_a), (world_coordinate - occ2_vertex_a))
    z6 = np.dot((occ2_vertex_c - occ2_vertex_b), (world_coordinate - occ2_vertex_b))
    z7 = np.dot((occ2_vertex_d - occ2_vertex_c), (world_coordinate - occ2_vertex_c))
    z8 = np.dot((occ2_vertex_a - occ2_vertex_d), (world_coordinate - occ2_vertex_d))
    is_positive1 = (z1 * z2 > 0) and (z3 * z4 > 0) and (z1 * z3 > 0)
    is_positive2 = (z5 * z6 > 0) and (z7 * z8 > 0) and (z5 * z7 > 0)
    if is_positive1 or is_positive2:
        image_coordinate = np.array([0, 0, 0])
    else:
        image_coordinate = world_coordinate
    return image_coordinate

#### 4.进行遮挡

In [9]:
def occlusion(data,data_left,data_right):
    # 寻找三个视角骨骼的边界范围
    maxX1, minX1, maxY1, minY1 = skel_boundary(data_left)
    maxX2, minX2, maxY2, minY2 = skel_boundary(data)
    maxX3, minX3, maxY3, minY3 = skel_boundary(data_right)
    # 左边视角取骨骼右半部分1/2,中间视角取骨骼左半部分1/3，右半部分1/3,右边视角取骨骼左半部分1/2
    occ1_minX, occ1_maxX = maxX2 - (maxX2 - minX2) /3, maxX1 - (maxX1 - minX1) / 2
    occ1_minY, occ1_maxY = min(minY1, minY2), max(maxY1, maxY2)
    occ2_minX, occ2_maxX = minX3 + (maxX3 - minX3) / 2, minX2 + (maxX2 - minX2) /3
    occ2_minY, occ2_maxY = min(minY2, minY3), max(maxY2, maxY3)
    occ1 = (occ1_maxX, occ1_minX, occ1_maxY, occ1_minY)
    occ2 = (occ2_maxX, occ2_minX, occ2_maxY, occ2_minY)
    # 将遮挡范围内的骨骼点置为0
    for m in range(2):
        for j in range(25):
            data_left[:,j,m] = judge_quadrilateral(data_left[:,j,m], occ1, occ2)
            data[:,j, m] = judge_quadrilateral(data[:,j, m], occ1, occ2)
            data_right[:,j, m] = judge_quadrilateral(data_right[:,j, m], occ1, occ2)
    return (data_left, data, data_right),(occ1,occ2)

#### 5.获得旋转数据

In [17]:
k_index = 8
t_index = 35
origin_data = data[k_index]
# 三个视角数据
skeleton_data = np.zeros((3,int(frame_nums[k_index]),25,2))
skeleton_data_left = np.zeros((3,int(frame_nums[k_index]),25,2))
skeleton_data_right = np.zeros((3,int(frame_nums[k_index]),25,2))
# 旋转操作
for t in range(int(frame_nums[k_index])):
    skeleton_data[:,t,:,:] = origin_data[:,t,:,:]
    skeleton_data_left[:,t,:,:] = rotation_xyz(origin_data[:,t,:,:],-math.pi/8)
    skeleton_data_right[:,t,:,:] = rotation_xyz(origin_data[:,t,:,:],math.pi/8)

#### 6.获得遮挡数据

In [18]:
# 三个视角数据
occ_skeleton_data = skeleton_data.copy()
occ_skeleton_data_left = skeleton_data_left.copy()
occ_skeleton_data_right = skeleton_data_right.copy()
# 两个遮挡区域
occ_area = np.zeros((2,int(frame_nums[k_index]),4))
for t in range(int(frame_nums[k_index])):
    (occ_skeleton_data_left[:,t,:,:],occ_skeleton_data[:,t,:,:],occ_skeleton_data_right[:,t,:,:]),(occ_area1, occ_area2) = occlusion(occ_skeleton_data[:,t,:,:],occ_skeleton_data_left[:,t,:,:],occ_skeleton_data_right[:,t,:,:])
    occ_area[0,t,:] = occ_area1
    occ_area[1,t,:] = occ_area2

#### 7.绘制原骨骼图

In [22]:
np.random.seed(202296)
# 骨骼连接关系
connecting_joint = np.array([2,1,21,3,21,5,6,7,21,9,10,11,1,13,14,15,1,17,18,19,2,23,8,25,12])
# 绘制画布
fig = plt.figure()
ax = Axes3D(fig)
ax.view_init(0, 180)
# 绘制3d骨骼点的每一帧
plt.cla()   # 清除当前画布的axes
# ax.set_xlim3d([0,5])
# ax.set_ylim3d([-1.2,1.8])
# ax.set_zlim3d([-0.7,1.2])

# 绘制遮挡区域
print(occ_area[0,t_index,:])
print(occ_area[1,t_index,:])
# occ1_x = np.arange(occ_area[0,t_index,1],occ_area[0,t_index,0],0.05)
# occ1_y = np.arange(occ_area[0,t_index,3],occ_area[0,t_index,2],0.05)
# occ1_x, occ1_y = np.meshgrid(occ1_x, occ1_y)
# ax.plot_surface(occ1_x*0+2.6,occ1_x,occ1_y,color='lime',alpha=0.2)

# occ2_x = np.arange(occ_area[1,t_index,1],occ_area[1,t_index,0],0.05)
# occ2_y = np.arange(occ_area[1,t_index,3],occ_area[1,t_index,2],0.05)
# occ2_x, occ2_y = np.meshgrid(occ2_x, occ2_y)
# ax.plot_surface(occ2_x*0+2.6,occ2_x,occ2_y,color='lime',alpha=0.2)

# 绘制第t_index帧里的第一个人
skeleton_data_xyz = skeleton_data[:,t_index,:,0]
# skeleton_data_xyz = occ_skeleton_data[:,t_index,:,0]
x = skeleton_data_xyz[0]
y = skeleton_data_xyz[1]
z = skeleton_data_xyz[2]
skeleton_data_xyz_left = skeleton_data_left[:,t_index,:,0]
# skeleton_data_xyz_left = occ_skeleton_data_left[:,t_index,:,0]
x_left = skeleton_data_xyz_left[0]
y_left = skeleton_data_xyz_left[1]
z_left = skeleton_data_xyz_left[2]
skeleton_data_xyz_right = skeleton_data_right[:,t_index,:,0]
# skeleton_data_xyz_right = occ_skeleton_data_right[:,t_index,:,0]
x_right = skeleton_data_xyz_right[0]
y_right = skeleton_data_xyz_right[1]
z_right = skeleton_data_xyz_right[2]

for v in range(25):
    col = (np.random.random(), np.random.random(), np.random.random())
    k = connecting_joint[v]-1
    ax.plot([z[v],z[k]],[x[v],x[k]],[y[v],y[k]], color=col, marker='o')   # 旋转前
    ax.plot([z_left[v],z_left[k]],[x_left[v],x_left[k]],[y_left[v],y_left[k]], color=col, marker='o')   # 向左旋转
    ax.plot([z_right[v],z_right[k]],[x_right[v],x_right[k]],[y_right[v],y_right[k]], color=col, marker='o')   # 向右旋转
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
ax.axis('off')
plt.savefig("pic/rotationFramework.svg",dpi=300,block=True)
# plt.show(block=True)

C:\Users\admin\AppData\Local\Temp/ipykernel_17784/465094903.py:6: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = Axes3D(fig)
C:\Users\admin\AppData\Local\Temp/ipykernel_17784/465094903.py:57: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "block" which is no longer supported as of 3.3 and will become an error in 3.6
  plt.savefig("pic/rotationFramework.svg",dpi=300,block=True)


[ 1.52837311  0.40312934  0.9020236  -0.5772794 ]
[ 0.21426248 -0.95087463  0.9020236  -0.5772794 ]


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelector

...

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\admin\Anaconda3\envs\UEX\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelector